In [19]:
!pip install tensorflow-gpu

In [20]:
# !unzip /content/ds.zip

Archive:  /content/ds.zip
  inflating: Fully Processed Datasets/ValidationOnly.tsv  
  inflating: Fully Processed Datasets/Ty.txt  
  inflating: Fully Processed Datasets/FinalTrainingOnly.tsv  
  inflating: Fully Processed Datasets/FinalTest.tsv  


In [41]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import csv
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Embedding ,Bidirectional
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import accuracy_score

In [42]:
d = pd.read_csv('/content/FinalTrainingOnly.tsv', sep="\t", header=None )

x=d[1]
y=d[2]
y=list(y)
x=list(x)

for i in range(len(x)):
  if type(x[i])==float:
    x[i]=" "
    y[i]= 1

y=np.array(y)

In [43]:
v = pd.read_csv('/content/ValidationOnly.tsv', sep="\t", header=None )

vx=v[1]
vy=v[2]
vy=list(vy)
vx=list(vx)

for i in range(len(vx)):
  if type(vx[i])==float:
    vx[i]=" "
    vy[i]= 1

vy=np.array(vy)

In [44]:
t = pd.read_csv('/content/FinalTest.tsv', sep="\t", header=None )
tx=list(t[1])
for i in range(len(tx)):
  if type(tx[i])==float:
    tx[i]=" "

In [45]:
ty = pd.read_csv('/content/Ty.txt', sep=",", header=None )
sentiment = []
for i in range(len(ty)):
    if ty[1][i]=='negative':
        sentiment.append(0)
    elif ty[1][i]=='neutral':
        sentiment.append(1)
    elif ty[1][i]=='positive':
        sentiment.append(2)

ty = np.array(sentiment)

In [46]:
tokenizer = Tokenizer(num_words=2500,split=' ')
tokenizer.fit_on_texts(x)

X= tokenizer.texts_to_sequences(x)
X = pad_sequences(X,maxlen=50)

VX= tokenizer.texts_to_sequences(vx)
VX = pad_sequences(VX,maxlen=50)


TX = tokenizer.texts_to_sequences(tx)
TX = pad_sequences(TX,maxlen=50)

In [47]:
vocab_size = 20000
embed_size = 128

In [48]:
# Model no 1 

ip = tf.keras.Input(shape=(X.shape[1]))
em = Embedding(vocab_size, embed_size,trainable=True)(ip)
x = Bidirectional(LSTM(128,activation='tanh', return_sequences=True, dropout=0.15, recurrent_dropout=0.15))(em) # Dimn shd be (None,200,128)
x = tf.keras.layers.Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)
avg_pool = tf.keras.layers.GlobalAveragePooling1D()(x)
max_pool = tf.keras.layers.GlobalMaxPool1D()(x)
x = tf.keras.layers.concatenate([avg_pool, max_pool])
x = Dense(3, activation='softmax')(x)
bi = tf.keras.Model(inputs = ip , outputs = x)
bi.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07), metrics=['accuracy'])
# tf.keras.utils.plot_model(bi, 'my_first_model.png')
history1 = bi.fit(X, y, epochs=7, batch_size=128,verbose=2,validation_data=(VX,vy))

Epoch 1/7
115/115 - 65s - loss: 0.9093 - accuracy: 0.5599 - val_loss: 0.8413 - val_accuracy: 0.5687
Epoch 2/7
115/115 - 65s - loss: 0.7812 - accuracy: 0.6470 - val_loss: 0.7313 - val_accuracy: 0.6967
Epoch 3/7
115/115 - 65s - loss: 0.7121 - accuracy: 0.6922 - val_loss: 0.6712 - val_accuracy: 0.7167
Epoch 4/7
115/115 - 64s - loss: 0.6571 - accuracy: 0.7205 - val_loss: 0.6533 - val_accuracy: 0.7210
Epoch 5/7
115/115 - 64s - loss: 0.5850 - accuracy: 0.7598 - val_loss: 0.5389 - val_accuracy: 0.8010
Epoch 6/7
115/115 - 64s - loss: 0.4865 - accuracy: 0.8050 - val_loss: 0.6173 - val_accuracy: 0.7770
Epoch 7/7
115/115 - 64s - loss: 0.4527 - accuracy: 0.8181 - val_loss: 0.4745 - val_accuracy: 0.8387


In [49]:
test_loss, test_acc = bi.evaluate(TX,ty)
print("Test accuracy of BiLSTM model = " + str(test_acc) )

94/94 [==============================] - 4s 41ms/step - loss: 1.0942 - accuracy: 0.6013
Test accuracy of BiLSTM model = 0.6013333201408386


In [50]:
# Model no 2 

ip = tf.keras.Input(shape=(X.shape[1]))
x = Embedding(vocab_size, embed_size,trainable=True)(ip)
x = tf.keras.layers.Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)
x = tf.keras.layers.GlobalMaxPool1D()(x)
x = tf.reshape(x,(-1,1,64))
x = LSTM(64,activation='tanh', dropout=0.15, recurrent_dropout=0.15)(x)
x = Dense(3, activation='softmax')(x)
cl = tf.keras.Model(inputs=ip, outputs=x)
cl.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07), metrics=['accuracy'])
history = cl.fit(X, y, epochs=5, batch_size=128,verbose=2,validation_data=(VX,vy))

Epoch 1/5
115/115 - 5s - loss: 0.8873 - accuracy: 0.5617 - val_loss: 0.7663 - val_accuracy: 0.6583
Epoch 2/5
115/115 - 5s - loss: 0.7616 - accuracy: 0.6575 - val_loss: 0.7355 - val_accuracy: 0.6597
Epoch 3/5
115/115 - 4s - loss: 0.6904 - accuracy: 0.7003 - val_loss: 0.6295 - val_accuracy: 0.7573
Epoch 4/5
115/115 - 4s - loss: 0.6098 - accuracy: 0.7564 - val_loss: 0.5665 - val_accuracy: 0.7877
Epoch 5/5
115/115 - 4s - loss: 0.5352 - accuracy: 0.7937 - val_loss: 0.5305 - val_accuracy: 0.8067


In [51]:
test_loss, test_acc = cl.evaluate(TX,ty)
print("Test accuracy = " + str(test_acc) )

94/94 [==============================] - 0s 4ms/step - loss: 0.9418 - accuracy: 0.6120
Test accuracy = 0.6119999885559082


In [52]:
# Model no 3

model = Sequential()
model.add(Embedding(vocab_size, embed_size, input_shape = (X.shape[1],)))
model.add(LSTM(units=264, activation='tanh'))
model.add(Dense(units=64,activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(units=3, activation='softmax'))

model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics = ['accuracy'])
history = model.fit(X, y, epochs=6, batch_size=128,verbose=2,validation_data=(VX,vy))

Epoch 1/6
115/115 - 7s - loss: 0.9193 - accuracy: 0.5383 - val_loss: 0.8049 - val_accuracy: 0.6327
Epoch 2/6
115/115 - 6s - loss: 0.7882 - accuracy: 0.6426 - val_loss: 0.7387 - val_accuracy: 0.6800
Epoch 3/6
115/115 - 6s - loss: 0.7297 - accuracy: 0.6843 - val_loss: 0.6965 - val_accuracy: 0.7017
Epoch 4/6
115/115 - 5s - loss: 0.6888 - accuracy: 0.7036 - val_loss: 0.6601 - val_accuracy: 0.7283
Epoch 5/6
115/115 - 5s - loss: 0.6452 - accuracy: 0.7277 - val_loss: 0.6344 - val_accuracy: 0.7467
Epoch 6/6
115/115 - 6s - loss: 0.5955 - accuracy: 0.7527 - val_loss: 0.5931 - val_accuracy: 0.7707


In [53]:
test_loss, test_acc = model.evaluate(TX,ty)
print("Test accuracy = " + str(test_acc) )

94/94 [==============================] - 1s 8ms/step - loss: 0.9188 - accuracy: 0.6063
Test accuracy = 0.606333315372467


In [54]:
# Model no 4

ip = tf.keras.Input(shape=(X.shape[1]))
b = Embedding(vocab_size, embed_size,trainable=True)(ip)
c1 = tf.keras.layers.Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(b)
c2 = tf.keras.layers.Conv1D(64, kernel_size=4, padding='valid', kernel_initializer='glorot_uniform')(b)
c3 = tf.keras.layers.Conv1D(64, kernel_size=5, padding='valid', kernel_initializer='glorot_uniform')(b)
x = tf.keras.layers.concatenate([c1, c2, c3],axis=1)
x = tf.keras.layers.GlobalMaxPool1D()(x)
x = tf.reshape(x,(-1,1,64))
x = Dense(64,activation='tanh')(x)
x = Dense(3, activation='softmax')(x)
cnn = tf.keras.Model(inputs=ip, outputs=x)
cnn.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07), metrics=['accuracy'])
cnn.summary()
history = cnn.fit(X, y, epochs=4, batch_size=128,verbose=2,validation_data=(VX,vy))

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 50, 128)      2560000     input_8[0][0]                    
__________________________________________________________________________________________________
conv1d_9 (Conv1D)               (None, 48, 64)       24640       embedding_10[0][0]               
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 47, 64)       32832       embedding_10[0][0]               
____________________________________________________________________________________________

In [55]:
test_loss, test_acc = cnn.evaluate(TX,ty)
print("Test accuracy = " + str(test_acc) )

94/94 [==============================] - 0s 5ms/step - loss: 1.0124 - accuracy: 0.6053
Test accuracy = 0.6053333282470703


In [56]:
x1 = bi.predict(TX)
x2 = cl.predict(TX)
x3 = model.predict(TX)
x4 = cnn.predict(TX)

In [57]:
x4=np.reshape(x4,(3000,3))

In [58]:
a=[]
for i in range(len(x1)):
  b=[]
  for j in range(len(x1[i])):
    z = max(x1[i][j],x2[i][j],x3[i][j],x4[i][j] )
    b.append(z)
  a.append(b)
  b=[]


In [59]:
a=np.array(a)
a=np.argmax(a,axis=1)

In [60]:
test_acc = accuracy_score(ty,a)
print("Test accuracy = " + str(test_acc) )

Test accuracy = 0.6386666666666667
